In [1]:
# ====================================================
# Configuration
# ====================================================
class CFG:
    debug=False
    apex=True
    print_freq=1000
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=10
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=16
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    hidden_size=256
    num_workers=4
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    label='LN_IC50'
    train=True

if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [2]:
# ====================================================
# Library
# ====================================================
import os
import gc
import math
import time
import random
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [3]:
# ====================================================
# Utils
# ====================================================
def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed=42)

In [4]:
# ====================================================
# Data Loading
# ====================================================
overlap_target = np.load("../data/overlap_target.npy",allow_pickle='TRUE').item()
use_tcols = ["Unnamed: 0"] + list(overlap_target.values()) 
ccle_expression = pd.read_csv("../data/CCLE_expression.csv", usecols=use_tcols)
ccle_expression.rename(columns={"Unnamed: 0": "DepMap_ID"}, inplace=True)
use_tcols = use_tcols[1:]
CFG.use_tcols = use_tcols
ccle_depmap_list = ccle_expression.DepMap_ID.values

gdsc = pd.read_csv("../processed_data/gdsc_overlap.csv")
gdsc = gdsc[gdsc.DepMap_ID.isin(ccle_depmap_list)]
gdsc = gdsc.merge(ccle_expression, on='DepMap_ID', how='left')
gdsc.reset_index(drop=True, inplace=True)

print(f"ccle_expression.shape:{ccle_expression.shape}")
display(ccle_expression.head())

print(f"gdsc.shape: {gdsc.shape}")
display(gdsc.head())

ccle_expression.shape:(1249, 2315)


,DepMap_ID,GCLC (2729),CFTR (1080),CYP51A1 (1595),BAD (572),LAP3 (51056),HS3ST1 (9957),AOC1 (26),M6PR (4074),TFPI (7035),SLC7A2 (6542),AK2 (204),NDUFAB1 (4706),PDK4 (5166),SLC22A16 (85413),ABCB5 (340273),SLC25A13 (10165),CALCR (799),SLC25A5 (292),LIG3 (3980),PRKAR2B (5577),CREBBP (1387),MPO (4353),PON1 (5444),CROT (54677),ABCB4 (5244),ITGAL (3683),PDK2 (5164),ITGA2B (3674),ABCC8 (6833),CACNA1G (8913),MAP3K9 (4293),RALA (5898),ALDH3B1 (221),TBXA2R (6915),UPP2 (151531),NOS2 (4843),SLC13A2 (9058),TRAPPC6A (79090),SCN4A (6329),CACNA2D2 (9254),SELE (6401),FMO3 (2328),PSMB1 (5689),SYN1 (6853),CAMK1G (57172),CYTH3 (9265),AASS (10157),MGST1 (4257),MMP25 (64386),MASP2 (10747),IYD (389434),UQCRC1 (7384),CD9 (928),NISCH (11188),SLC6A13 (6540),PRSS3 (5646),CD4 (920),BTK (695),SCMH1 (22955),FYN (2534),FMO1 (2326),SLC6A7 (6534),LARS2 (23395),PLAUR (5329),DCN (1634),GABRA3 (2556),CD22 (933),LTF (4057),NR1H4 (9971),ALOX5 (240),KDM5D (8284),CLK1 (1195),GPRC5A (9052),POLA2 (23649),CAPN1 (823),ACPP (55),MDH1 (4190),WAS (7454),DPEP1 (1800),NPC1L1 (29881),STMN4 (81551),CHDH (55349),ATP2C1 (27032),IGF1 (3479),RALBP1 (10928),CYP24A1 (1591),HGF (3082),CYP3A43 (64816),SLC7A9 (11136),CPS1 (1373),GABRA1 (2554),NDUFS1 (4719),ALAS1 (211),GLRX2 (51022),ABCC2 (1244),GCLM (2730),HSD17B6 (8630),NR1H3 (10062),TYMP (1890),CD44 (960),IFNGR1 (3459),TNFRSF1B (7133),POU2F2 (5452),SARS (6301),SLC4A7 (9497),ADSS (159),OTC (5009),SLC18A1 (6570),CASR (846),RPL26L1 (51121),FLT4 (2324),MAT2B (27430),VCAN (1462),CTNS (1497),PSMA4 (5685),PARP3 (10039),RETSAT (54884),ADAM28 (10863),ADRB1 (153),HSPA5 (3309),CP (1356),MAP4 (4134),HDAC9 (9734),RRM2B (50484),GUCA1A (2978),H6PD (9563),ELN (2006),HEXB (3074),SLC4A8 (9498),PTGER3 (5733),MAPK9 (5601),RAD51 (5888),PIK3CB (5291),PRSS8 (5652),MSMO1 (6307),KCNQ1 (3784),TRAPPC3 (27095),KCNH2 (3757),SERPINB3 (6317),SOAT1 (6646),F7 (2155),CAMK2B (816),DGKG (1608),TBXAS1 (6916),ALDH18A1 (5832),ACAA1 (30),BCAT1 (586),HDAC7 (51564),TNK2 (10188),CS (1431),CA11 (770),EPN1 (29924),HAGH (3029),SLC12A2 (6558),OAT (4942),GLP2R (9340),DGKA (1606),KARS (3735),NDUFB4 (4710),SLK (9748),GSTO2 (119391),PRKCQ (5588),ME1 (4199),MTHFD2 (10797),PDE4A (5141),FGFR2 (2263),FECH (2235),IDI1 (3422),TNFRSF1A (7132),CACNB1 (782),PKM (5315),RHOA (387),ZFY (7544),IARS2 (55699),IDH3G (3421),ROCK1 (6093),PDK3 (5165),HDAC4 (9759),FGFR3 (2261),ACSL4 (2182),PYGM (5837),ABCC9 (10060),MAOB (4129),RORA (6095),DRD4 (1815),GUCY2C (2984),PFN2 (5217),DAPP1 (27071),SLC44A1 (23446),SMG6 (23293),POLB (5423),NDST1 (3340),ASNS (440),CHAT (1103),CSNK2A2 (1459),CAMK2A (815),CDC42 (998),SLC12A3 (6559),RDH11 (51109),PRKACA (5566),ALDH3A2 (224),HSD17B10 (3028),HMMR (3161),P4HA2 (8974),FCGR2B (2213),PVR (5817),SCARB1 (949),PDE8A (5151),SDHA (6389),ABCB11 (8647),PTGS2 (5743),NSF (4905),CA12 (771),LMAN1 (3998),SLC12A1 (6557),TUBE1 (51175),KCNQ2 (3785),ACAT1 (38),FGF4 (2249),MOCOS (55034),PLD1 (5337),RAB7A (7879),UNG (7374),SLC46A1 (113235),ACACB (32),NT5C2 (22978),DGKD (8527),RARB (5915),TOP2B (7155),NFKB2 (4791),IL4R (3566),SNRPA (6626),TYR (7299),FGFR1 (2260),MAP2 (4133),ARAF (369),MCCC1 (56922),PIK3C3 (5289),ADCY2 (108),PPP2R5C (5527),GNB1 (2782),EDN1 (1906),VDAC3 (7419),SLC1A3 (6507),CEACAM1 (634),FDFT1 (2222),PAFAH1B3 (5050),PGM1 (5236),KEAP1 (9817),...,SLC38A3 (10991),PPP3R2 (5535),PDCD1 (5133),CHM (1121),SERPINA5 (5104),LCTL (197021),DPYD (1806),SLC4A5 (57835),ADRB3 (155),PLA2G2E (30814),CNR2 (1269),NDUFA4 (4697),ALKBH2 (121642),S100A13 (6284),MAOA (4128),FHIT (2272),HMGB1 (3146),HLA-DRB1 (3123),AKR1C3 (8644),S100A4 (6275),ACADSB (36),RYR1 (6261),KRTAP5-3 (387266),TUBB (203068),PPIA (5478),GRM7 (2917),ATP2A1 (487),IARS (3376),ADH7 (131),CD55 (1604),PTPN1 (5770),NOXO1 (124056),EPHB4 (2050),PRTN3 (5657),ASMT (438),MYL6B (140465),ESRRG (2104),SULT1A1 (6817),TPK1 (27010),SLC6A9 (6536),MME (4311),CACNA1H (8912),HDAC2 (3066),MMP1 (4312),ADH1B (125),UGT2B15 (7366),HRH1 (3269),TRAPPC4 (51399),TLR7 (51284),TRPV1 (7442),VKORC1L1 (154807),DAPK1 (1612),GM2A (2760),S100A2 (6273),CHRNG (1146),ADA 

gdsc.shape: (58912, 2320)


,DRUG_NAME,DRUG_ID,pubchem,SMILES,DepMap_ID,LN_IC50,GCLC (2729),CFTR (1080),CYP51A1 (1595),BAD (572),LAP3 (51056),HS3ST1 (9957),AOC1 (26),M6PR (4074),TFPI (7035),SLC7A2 (6542),AK2 (204),NDUFAB1 (4706),PDK4 (5166),SLC22A16 (85413),ABCB5 (340273),SLC25A13 (10165),CALCR (799),SLC25A5 (292),LIG3 (3980),PRKAR2B (5577),CREBBP (1387),MPO (4353),PON1 (5444),CROT (54677),ABCB4 (5244),ITGAL (3683),PDK2 (5164),ITGA2B (3674),ABCC8 (6833),CACNA1G (8913),MAP3K9 (4293),RALA (5898),ALDH3B1 (221),TBXA2R (6915),UPP2 (151531),NOS2 (4843),SLC13A2 (9058),TRAPPC6A (79090),SCN4A (6329),CACNA2D2 (9254),SELE (6401),FMO3 (2328),PSMB1 (5689),SYN1 (6853),CAMK1G (57172),CYTH3 (9265),AASS (10157),MGST1 (4257),MMP25 (64386),MASP2 (10747),IYD (389434),UQCRC1 (7384),CD9 (928),NISCH (11188),SLC6A13 (6540),PRSS3 (5646),CD4 (920),BTK (695),SCMH1 (22955),FYN (2534),FMO1 (2326),SLC6A7 (6534),LARS2 (23395),PLAUR (5329),DCN (1634),GABRA3 (2556),CD22 (933),LTF (4057),NR1H4 (9971),ALOX5 (240),KDM5D (8284),CLK1 (1195),GPRC5A (9052),POLA2 (23649),CAPN1 (823),ACPP (55),MDH1 (4190),WAS (7454),DPEP1 (1800),NPC1L1 (29881),STMN4 (81551),CHDH (55349),ATP2C1 (27032),IGF1 (3479),RALBP1 (10928),CYP24A1 (1591),HGF (3082),CYP3A43 (64816),SLC7A9 (11136),CPS1 (1373),GABRA1 (2554),NDUFS1 (4719),ALAS1 (211),GLRX2 (51022),ABCC2 (1244),GCLM (2730),HSD17B6 (8630),NR1H3 (10062),TYMP (1890),CD44 (960),IFNGR1 (3459),TNFRSF1B (7133),POU2F2 (5452),SARS (6301),SLC4A7 (9497),ADSS (159),OTC (5009),SLC18A1 (6570),CASR (846),RPL26L1 (51121),FLT4 (2324),MAT2B (27430),VCAN (1462),CTNS (1497),PSMA4 (5685),PARP3 (10039),RETSAT (54884),ADAM28 (10863),ADRB1 (153),HSPA5 (3309),CP (1356),MAP4 (4134),HDAC9 (9734),RRM2B (50484),GUCA1A (2978),H6PD (9563),ELN (2006),HEXB (3074),SLC4A8 (9498),PTGER3 (5733),MAPK9 (5601),RAD51 (5888),PIK3CB (5291),PRSS8 (5652),MSMO1 (6307),KCNQ1 (3784),TRAPPC3 (27095),KCNH2 (3757),SERPINB3 (6317),SOAT1 (6646),F7 (2155),CAMK2B (816),DGKG (1608),TBXAS1 (6916),ALDH18A1 (5832),ACAA1 (30),BCAT1 (586),HDAC7 (51564),TNK2 (10188),CS (1431),CA11 (770),EPN1 (29924),HAGH (3029),SLC12A2 (6558),OAT (4942),GLP2R (9340),DGKA (1606),KARS (3735),NDUFB4 (4710),SLK (9748),GSTO2 (119391),PRKCQ (5588),ME1 (4199),MTHFD2 (10797),PDE4A (5141),FGFR2 (2263),FECH (2235),IDI1 (3422),TNFRSF1A (7132),CACNB1 (782),PKM (5315),RHOA (387),ZFY (7544),IARS2 (55699),IDH3G (3421),ROCK1 (6093),PDK3 (5165),HDAC4 (9759),FGFR3 (2261),ACSL4 (2182),PYGM (5837),ABCC9 (10060),MAOB (4129),RORA (6095),DRD4 (1815),GUCY2C (2984),PFN2 (5217),DAPP1 (27071),SLC44A1 (23446),SMG6 (23293),POLB (5423),NDST1 (3340),ASNS (440),CHAT (1103),CSNK2A2 (1459),CAMK2A (815),CDC42 (998),SLC12A3 (6559),RDH11 (51109),PRKACA (5566),ALDH3A2 (224),HSD17B10 (3028),HMMR (3161),P4HA2 (8974),FCGR2B (2213),PVR (5817),SCARB1 (949),PDE8A (5151),SDHA (6389),ABCB11 (8647),PTGS2 (5743),NSF (4905),CA12 (771),LMAN1 (3998),SLC12A1 (6557),TUBE1 (51175),KCNQ2 (3785),ACAT1 (38),FGF4 (2249),MOCOS (55034),PLD1 (5337),RAB7A (7879),UNG (7374),SLC46A1 (113235),ACACB (32),NT5C2 (22978),DGKD (8527),RARB (5915),TOP2B (7155),NFKB2 (4791),IL4R (3566),SNRPA (6626),TYR (7299),FGFR1 (2260),MAP2 (4133),ARAF (369),MCCC1 (56922),PIK3C3 (5289),ADCY2 (108),PPP2R5C (5527),GNB1 (2782),EDN1 (1906),VDAC3 (7419),SLC1A3 (6507),...,SLC38A3 (10991),PPP3R2 (5535),PDCD1 (5133),CHM (1121),SERPINA5 (5104),LCTL (197021),DPYD (1806),SLC4A5 (57835),ADRB3 (155),PLA2G2E (30814),CNR2 (1269),NDUFA4 (4697),ALKBH2 (121642),S100A13 (6284),MAOA (4128),FHIT (2272),HMGB1 (3146),HLA-DRB1 (3123),AKR1C3 (8644),S100A4 (6275),ACADSB (36),RYR1 (6261),KRTAP5-3 (387266),TUBB (203068),PPIA (5478),GRM7 (2917),ATP2A1 (487),IARS (3376),ADH7 (131),CD55 (1604),PTPN1 (5770),NOXO1 (124056),EPHB4 (2050),PRTN3 (5657),ASMT (438),MYL6B (140465),ESRRG (2104),SULT1A1 (6817),TPK1 (27010),SLC6A9 (6536),MME (4311),CACNA1H (8912),HDAC2 (3066),MMP1 (4312),ADH1B (125),UGT2B15 (7366),HRH1 (3269),TRAPPC4 (51399),TLR7 (51284),TRPV1 (7442),VKORC1L1 (154807),DAPK1 (1612),GM2A (2760),S100A2 (6273),CHRNG (1146),ADA (100),SCN8A (6334),IGF2R (3

In [5]:
# ====================================================
# CV split
# ====================================================
Fold = GroupKFold(n_splits=CFG.n_fold)
groups = gdsc["DRUG_NAME"].values
for n, (train_index, val_index) in enumerate(Fold.split(gdsc, gdsc[CFG.label], groups)):
    gdsc.loc[val_index, 'fold'] = int(n)
gdsc['fold'] = gdsc['fold'].astype(int)
display(gdsc.groupby('fold').size())

fold
0    11725
1    11733
2    11654
3    11918
4    11882
dtype: int64

In [6]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

In [7]:
# ====================================================
# Dataset
# ====================================================
class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.drug_embedding = df["SMILES"].values
        self.target_embedding = df[cfg.use_tcols].values.astype(np.float32)
        self.label = df[cfg.label].values.astype(np.float16)
    
    def __len__(self):
        return len(self.drug_embedding)
    
    def __getitem__(self, item):
        drug_embedding = self.smiles2morgan(self.drug_embedding[item])
        target_embedding = self.target_embedding[item]
        label = self.label[item]
        return (drug_embedding, target_embedding), label
    
    def smiles2morgan(self, s, radius=2, nBits=512):
        try:
            mol = Chem.MolFromSmiles(s)
            features_vec = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)
            features = np.zeros((1,))
            DataStructs.ConvertToNumpyArray(features_vec, features)
        except:
            print('rdkit not found this smiles for morgan: ' + s + ' convert to all 0 features')
            features = np.zeros((nBits, ))
        return features.astype(np.float32)

In [8]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.drug_fc = nn.Linear(512, cfg.hidden_size)
        self.target_fc = nn.Linear(len(cfg.use_tcols), cfg.hidden_size)
        self.fc = nn.Linear(cfg.hidden_size*2, 1)
        self.init_layers()
    
    def init_layers(self):
        self._init_weights(self.drug_fc)
        self._init_weights(self.target_fc)
        self._init_weights(self.fc)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform(m.weight)
            m.bias.data.fill_(0.01)
    
    def forward(self, inputs):
        drug_embedding, target_embedding = inputs
        drug_features = F.relu(self.drug_fc(drug_embedding))
        target_features = F.relu(self.target_fc(target_embedding))
        combined_features = torch.cat((drug_features, target_features), axis=1)
        output = self.fc(combined_features)
        return output

In [9]:
def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (inputs, labels) in enumerate(train_loader):
        for k, v in enumerate(inputs):
            inputs[k] = inputs[k].to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs)
        loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))
        loss = loss.reshape([-1]).mean()
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
    return losses.avg

def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        for k, v in enumerate(inputs):
            inputs[k] = inputs[k].to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(inputs)
        loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))
        loss = loss.reshape([-1]).mean()
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

In [10]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")
    
    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    
    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)
    
    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    
    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG)
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler=='linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler=='cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)
    
    # ====================================================
    # loop
    # ====================================================
    criterion = nn.MSELoss(reduction="none")
    
    for epoch in range(CFG.epochs):
        
        start_time = time.time()
        
        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)
        
        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)
        
        elapsed = time.time() - start_time
        
        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        
    valid_folds['oof'] = predictions
    
    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [11]:
if __name__ == '__main__':
    
    def get_result(oof_df):
        labels = oof_df[CFG.label].values
        preds = oof_df['oof'].values
        mse_loss = mean_squared_error(labels, preds)
        LOGGER.info(f'MSE loss: {mse_loss:<.4f}')
    
    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(gdsc, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
    
    LOGGER.info(f"========== All finished ==========")

========== fold: 0 training ==========


Epoch: [1][0/2949] Elapsed 0m 0s (remain 6m 58s) Loss: 13.6719(13.6719) Grad: nan  LR: 0.00002000  
Epoch: [1][1000/2949] Elapsed 0m 2s (remain 0m 5s) Loss: 5.5938(9.4401) Grad: 247993.1250  LR: 0.00001994  
Epoch: [1][2000/2949] Elapsed 0m 5s (remain 0m 2s) Loss: 5.5586(6.9860) Grad: 84544.9141  LR: 0.00001977  
Epoch: [1][2948/2949] Elapsed 0m 7s (remain 0m 0s) Loss: 2.3184(5.7990) Grad: 99838.9766  LR: 0.00001951  
EVAL: [0/733] Elapsed 0m 0s (remain 1m 6s) Loss: 1.1146(1.1146) 


Epoch 1 - avg_train_loss: 5.7990  avg_val_loss: 4.1740  time: 8s


EVAL: [732/733] Elapsed 0m 1s (remain 0m 0s) Loss: 3.8097(4.1740) 
Epoch: [2][0/2949] Elapsed 0m 0s (remain 4m 40s) Loss: 2.1836(2.1836) Grad: nan  LR: 0.00001951  
Epoch: [2][1000/2949] Elapsed 0m 2s (remain 0m 5s) Loss: 1.8135(2.9320) Grad: 44055.4258  LR: 0.00001913  
Epoch: [2][2000/2949] Elapsed 0m 5s (remain 0m 2s) Loss: 1.8066(2.7772) Grad: 54690.6797  LR: 0.00001864  
Epoch: [2][2948/2949] Elapsed 0m 7s (remain 0m 0s) Loss: 2.4121(2.6965) Grad: 42253.5898  LR: 0.00001809  
EVAL: [0/733] Elapsed 0m 0s (remain 1m 7s) Loss: 1.2761(1.2761) 


Epoch 2 - avg_train_loss: 2.6965  avg_val_loss: 3.7227  time: 9s


...

Epoch 9 - avg_train_loss: 2.0052  avg_val_loss: 4.7298  time: 9s


EVAL: [742/743] Elapsed 0m 1s (remain 0m 0s) Loss: 4.2574(4.7298) 
Epoch: [10][0/2939] Elapsed 0m 0s (remain 4m 52s) Loss: 0.6094(0.6094) Grad: nan  LR: 0.00000049  
Epoch: [10][1000/2939] Elapsed 0m 2s (remain 0m 5s) Loss: 2.8906(1.9990) Grad: 253092.4844  LR: 0.00000021  
Epoch: [10][2000/2939] Elapsed 0m 5s (remain 0m 2s) Loss: 1.8389(1.9868) Grad: 28959.0332  LR: 0.00000005  
Epoch: [10][2938/2939] Elapsed 0m 7s (remain 0m 0s) Loss: 1.1162(1.9927) Grad: 68507.9375  LR: 0.00000000  
EVAL: [0/743] Elapsed 0m 0s (remain 1m 9s) Loss: 1.0958(1.0958) 


Epoch 10 - avg_train_loss: 1.9927  avg_val_loss: 4.7376  time: 9s


EVAL: [742/743] Elapsed 0m 1s (remain 0m 0s) Loss: 4.2533(4.7376) 


========== fold: 4 result ==========
MSE loss: 4.7376
========== CV ==========
MSE loss: 7.4703
========== All finished ==========
